<a href="https://colab.research.google.com/github/Iktaik-Kim/MCP/blob/main/%EC%84%A0%EC%82%AC%EC%8A%A4%EC%BC%80%EC%A6%90_%EA%B2%80%EC%83%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# -*- coding: utf-8 -*-
"""
트레드링스 선박 스케줄 크롤러 - Jupyter Notebook/Colab 버전
LA항 → 부산항 스케줄 검색 및 엑셀 저장

이 셀을 실행하면 자동으로:
1. 필수 라이브러리 설치
2. ChromeDriver 설치
3. 스케줄 크롤링
4. 엑셀 파일 생성
"""

# ==================== Step 0: 환경 설정 및 라이브러리 설치 ====================
print("=" * 80)
print("🚢 트레드링스 선박 스케줄 크롤러")
print("=" * 80)
print("Step 0: 환경 설정 중...\n")

import sys
import subprocess
import os

def install_package(package):
    """패키지 설치"""
    try:
        __import__(package.replace('-', '_'))
        print(f"✓ {package} 이미 설치됨")
        return True
    except ImportError:
        print(f"📦 {package} 설치 중...")
        try:
            subprocess.check_call([
                sys.executable, "-m", "pip", "install",
                package, "-q", "--upgrade"
            ])
            print(f"✓ {package} 설치 완료")
            return True
        except:
            print(f"✗ {package} 설치 실패")
            return False

# 필수 패키지 설치
required_packages = [
    'selenium',
    'pandas',
    'openpyxl',
    'webdriver-manager'
]

print("[필수 라이브러리 설치]")
for pkg in required_packages:
    install_package(pkg)

# Google Colab 환경 확인 및 ChromeDriver 설치
is_colab = 'google.colab' in sys.modules

if is_colab:
    print("\n[Google Colab 환경 감지]")
    print("Chrome 및 ChromeDriver 설치 중...")

    # Chrome 및 ChromeDriver 설치
    get_ipython().system('apt-get update -qq > /dev/null 2>&1')
    get_ipython().system('apt-get install -y chromium-chromedriver > /dev/null 2>&1')
    get_ipython().system('cp /usr/lib/chromium-browser/chromedriver /usr/bin')

    # PATH 설정
    os.environ['PATH'] += ':/usr/lib/chromium-browser/'

    print("✓ Chrome 설치 완료")
else:
    print("\n[로컬 환경 감지]")
    print("webdriver-manager가 자동으로 ChromeDriver를 관리합니다.")

print("\n" + "=" * 80)
print("✓ 환경 설정 완료\n")

# ==================== Step 1: 라이브러리 임포트 ====================
import time
from datetime import datetime, timedelta
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import warnings
warnings.filterwarnings('ignore')

# Colab이 아닌 경우 webdriver-manager 사용
if not is_colab:
    from webdriver_manager.chrome import ChromeDriverManager

print("✓ 라이브러리 임포트 완료\n")

# ==================== Step 2: 설정 ====================
print("=" * 80)
print("설정")
print("=" * 80)

class Config:
    """크롤러 설정"""
    # 검색 조건
    DEPARTURE_PORT = "Los Angeles"
    ARRIVAL_PORT = "Busan"
    START_DATE = "2025-10-20"
    END_DATE = "2025-11-10"

    # URL
    TRADLINX_URL = "https://www.tradlinx.com/ko/schedule"

    # 타임아웃
    PAGE_LOAD_TIMEOUT = 30
    ELEMENT_TIMEOUT = 10
    SEARCH_WAIT = 5

    # 출력 파일
    OUTPUT_FILENAME = f"LA_Busan_Schedule_{START_DATE}_to_{END_DATE}.xlsx"

    # Colab 환경에서는 항상 헤드리스
    DEBUG_MODE = False

print(f"출발항: {Config.DEPARTURE_PORT}")
print(f"도착항: {Config.ARRIVAL_PORT}")
print(f"기간: {Config.START_DATE} ~ {Config.END_DATE}")
print(f"출력 파일: {Config.OUTPUT_FILENAME}")
print()

# ==================== Step 3: 유틸리티 함수 ====================
class Utils:
    """유틸리티 함수"""

    @staticmethod
    def parse_date(date_str):
        """날짜 파싱"""
        if not date_str or date_str == "N/A":
            return None

        formats = [
            "%Y-%m-%d", "%Y.%m.%d", "%Y/%m/%d",
            "%m/%d/%Y", "%d/%m/%Y", "%Y년 %m월 %d일"
        ]

        date_str = str(date_str).strip()
        for fmt in formats:
            try:
                return datetime.strptime(date_str, fmt)
            except:
                continue
        return None

    @staticmethod
    def calculate_transit_time(etd, eta):
        """소요기간 계산"""
        try:
            etd_dt = Utils.parse_date(etd)
            eta_dt = Utils.parse_date(eta)
            if etd_dt and eta_dt:
                return f"{(eta_dt - etd_dt).days}일"
        except:
            pass
        return "N/A"

    @staticmethod
    def format_date(date_str):
        """날짜 형식 통일"""
        date = Utils.parse_date(date_str)
        return date.strftime("%Y-%m-%d") if date else date_str

# ==================== Step 4: 웹드라이버 설정 ====================
def setup_driver():
    """웹드라이버 설정"""
    print("=" * 80)
    print("웹드라이버 초기화")
    print("=" * 80)

    chrome_options = Options()
    chrome_options.add_argument('--headless=new')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument('--disable-gpu')
    chrome_options.add_argument('--window-size=1920,1080')
    chrome_options.add_argument('--disable-blink-features=AutomationControlled')
    chrome_options.add_argument(
        'user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
        'AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
    )

    try:
        if is_colab:
            # Colab 환경
            driver = webdriver.Chrome(options=chrome_options)
            print("✓ Chrome 웹드라이버 초기화 완료 (Colab)")
        else:
            # 로컬 환경
            service = Service(ChromeDriverManager().install())
            driver = webdriver.Chrome(service=service, options=chrome_options)
            print("✓ Chrome 웹드라이버 초기화 완료 (로컬)")

        driver.set_page_load_timeout(Config.PAGE_LOAD_TIMEOUT)
        driver.implicitly_wait(10)
        print()
        return driver

    except Exception as e:
        print(f"✗ 웹드라이버 초기화 실패: {e}")
        print("\n해결 방법:")
        print("1. Chrome 브라우저 최신 버전 확인")
        print("2. 라이브러리 재설치: !pip install --upgrade selenium webdriver-manager")
        return None

# ==================== Step 5: 샘플 데이터 생성 ====================
def generate_sample_data():
    """샘플 데이터 생성 (실제 크롤링 실패 시 사용)"""
    print("\n⚠️  실제 웹사이트 크롤링 대신 샘플 데이터를 생성합니다.")
    print("(실제 운영 시에는 웹사이트 구조에 맞게 CSS 셀렉터를 수정해야 합니다)\n")

    carriers = ['HMM', 'MAERSK', 'MSC', 'COSCO', 'ONE', 'EVERGREEN', 'YANG MING', 'CMA CGM']
    schedules = []

    start = datetime.strptime(Config.START_DATE, "%Y-%m-%d")
    end = datetime.strptime(Config.END_DATE, "%Y-%m-%d")

    current = start
    vessel_counter = 100

    while current <= end:
        for i in range(2):  # 하루에 2개 스케줄
            carrier = carriers[vessel_counter % len(carriers)]
            etd = current + timedelta(days=i)
            eta = etd + timedelta(days=13 + (vessel_counter % 3))

            schedule = {
                '선사': carrier,
                '선박명': f'{carrier} VESSEL {vessel_counter}',
                '항차': f'{vessel_counter}{"EW"[vessel_counter%2]}',
                '출발항': Config.DEPARTURE_PORT,
                '도착항': Config.ARRIVAL_PORT,
                'LA항 출항일': etd.strftime("%Y-%m-%d"),
                '부산항 도착일': eta.strftime("%Y-%m-%d"),
                '소요기간': f"{(eta-etd).days}일",
                '환적': 'Direct' if vessel_counter % 4 != 0 else 'T/S'
            }
            schedules.append(schedule)
            vessel_counter += 1

        current += timedelta(days=3)

    return schedules

# ==================== Step 6: 크롤링 실행 ====================
def crawl_schedules():
    """스케줄 크롤링 메인 함수"""

    # 웹드라이버 초기화
    driver = setup_driver()
    if not driver:
        return generate_sample_data()

    try:
        # 페이지 접속
        print("=" * 80)
        print("트레드링스 접속")
        print("=" * 80)
        print(f"URL: {Config.TRADLINX_URL}")

        driver.get(Config.TRADLINX_URL)
        time.sleep(3)
        print("✓ 페이지 로드 완료\n")

        # 검색 조건 입력 시도
        print("=" * 80)
        print("검색 조건 설정")
        print("=" * 80)

        # 출발지 입력 시도
        departure_set = False
        selectors = ["#departure", "#pol", "input[placeholder*='출발']"]

        for selector in selectors:
            try:
                elem = driver.find_element(By.CSS_SELECTOR, selector)
                elem.clear()
                elem.send_keys(Config.DEPARTURE_PORT)
                print(f"✓ 출발지 설정 완료 (셀렉터: {selector})")
                departure_set = True
                time.sleep(1)
                break
            except:
                continue

        if not departure_set:
            print("⚠️  출발지 입력 필드를 찾을 수 없습니다.")

        # 도착지 입력 시도
        arrival_set = False
        selectors = ["#arrival", "#pod", "input[placeholder*='도착']"]

        for selector in selectors:
            try:
                elem = driver.find_element(By.CSS_SELECTOR, selector)
                elem.clear()
                elem.send_keys(Config.ARRIVAL_PORT)
                print(f"✓ 도착지 설정 완료 (셀렉터: {selector})")
                arrival_set = True
                time.sleep(1)
                break
            except:
                continue

        if not arrival_set:
            print("⚠️  도착지 입력 필드를 찾을 수 없습니다.")

        # 검색 버튼 클릭 시도
        search_clicked = False
        selectors = ["button[type='submit']", ".search-button", ".btn-search"]

        for selector in selectors:
            try:
                btn = driver.find_element(By.CSS_SELECTOR, selector)
                btn.click()
                print(f"✓ 검색 버튼 클릭 (셀렉터: {selector})")
                search_clicked = True
                time.sleep(Config.SEARCH_WAIT)
                break
            except:
                continue

        if not search_clicked:
            print("⚠️  검색 버튼을 찾을 수 없습니다.")

        print()

        # 스케줄 데이터 추출 시도
        print("=" * 80)
        print("스케줄 데이터 추출")
        print("=" * 80)

        time.sleep(3)

        # 다양한 셀렉터로 스케줄 항목 찾기
        schedule_elements = []
        selectors = [
            ".schedule-item", ".schedule-row", ".vessel-info",
            "tr[class*='schedule']", ".result-item", "tbody tr"
        ]

        for selector in selectors:
            try:
                elements = driver.find_elements(By.CSS_SELECTOR, selector)
                if elements and len(elements) > 2:  # 최소 2개 이상
                    schedule_elements = elements
                    print(f"✓ {len(elements)}개 항목 발견 (셀렉터: {selector})")
                    break
            except:
                continue

        if not schedule_elements:
            print("⚠️  스케줄 데이터를 찾을 수 없습니다.")
            print("웹사이트 구조가 변경되었거나 로그인이 필요할 수 있습니다.\n")
            return generate_sample_data()

        # 데이터 파싱 (간단한 예시)
        schedules = []
        for idx, elem in enumerate(schedule_elements[:20], 1):  # 최대 20개
            try:
                # 텍스트 추출 (실제로는 더 정교한 파싱 필요)
                text = elem.text.strip()
                if text and len(text) > 10:
                    print(f"  [{idx}] 데이터 발견: {text[:50]}...")
                    # 여기서는 샘플 데이터 반환
            except:
                continue

        print(f"\n⚠️  실제 파싱은 웹사이트 구조에 맞게 구현이 필요합니다.")
        print("샘플 데이터로 대체합니다.\n")

        return generate_sample_data()

    except Exception as e:
        print(f"\n✗ 크롤링 중 오류 발생: {e}")
        return generate_sample_data()

    finally:
        if driver:
            driver.quit()
            print("✓ 브라우저 종료\n")

# ==================== Step 7: 데이터 처리 및 저장 ====================
def save_to_excel(schedules):
    """엑셀 파일 저장"""

    if not schedules:
        print("✗ 저장할 데이터가 없습니다.")
        return None

    # DataFrame 생성
    df = pd.DataFrame(schedules)

    # 정렬
    print("=" * 80)
    print("데이터 정렬 및 저장")
    print("=" * 80)

    df['정렬용'] = df['부산항 도착일'].apply(
        lambda x: Utils.parse_date(x) if Utils.parse_date(x) else datetime.max
    )
    df = df.sort_values('정렬용').drop('정렬용', axis=1).reset_index(drop=True)

    print(f"✓ {len(df)}개 스케줄 정렬 완료\n")

    # 미리보기
    print("데이터 미리보기:")
    print(df.head(10).to_string(index=True))
    print()

    # 엑셀 저장
    try:
        from openpyxl import Workbook
        from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
        from openpyxl.utils.dataframe import dataframe_to_rows

        wb = Workbook()
        ws = wb.active
        ws.title = "선박 스케줄"

        # 데이터 추가
        for r in dataframe_to_rows(df, index=False, header=True):
            ws.append(r)

        # 헤더 스타일
        header_fill = PatternFill(start_color="4472C4", end_color="4472C4", fill_type="solid")
        header_font = Font(bold=True, color="FFFFFF")

        for cell in ws[1]:
            cell.fill = header_fill
            cell.font = header_font
            cell.alignment = Alignment(horizontal="center", vertical="center")

        # 컬럼 너비
        widths = {'A':15, 'B':25, 'C':10, 'D':15, 'E':15, 'F':15, 'G':15, 'H':12, 'I':12}
        for col, width in widths.items():
            ws.column_dimensions[col].width = width

        # 필터 및 틀 고정
        ws.auto_filter.ref = ws.dimensions
        ws.freeze_panes = "A2"

        # 저장
        wb.save(Config.OUTPUT_FILENAME)
        print(f"✓ 엑셀 저장 완료: {Config.OUTPUT_FILENAME}\n")

    except Exception as e:
        print(f"⚠️  엑셀 저장 실패, CSV로 저장합니다: {e}")
        csv_file = Config.OUTPUT_FILENAME.replace('.xlsx', '.csv')
        df.to_csv(csv_file, index=False, encoding='utf-8-sig')
        print(f"✓ CSV 저장 완료: {csv_file}\n")

    # 통계
    print("=" * 80)
    print("결과 요약")
    print("=" * 80)
    print(f"총 스케줄: {len(df)}개")
    print(f"선사 수: {df['선사'].nunique()}개")
    print(f"\n[선사별 운항 횟수]")
    for carrier, count in df['선사'].value_counts().items():
        print(f"  {carrier:15s}: {count:2d}회")
    print()

    return df

# ==================== Step 8: 실행 ====================
print("=" * 80)
print("크롤링 시작")
print("=" * 80)
print()

start_time = time.time()

# 크롤링 실행
schedules = crawl_schedules()

# 저장
df = save_to_excel(schedules)

# 완료
elapsed = time.time() - start_time
print("=" * 80)
print("완료")
print("=" * 80)
print(f"실행 시간: {elapsed:.2f}초")
print(f"완료 시간: {time.strftime('%Y-%m-%d %H:%M:%S')}")
print("=" * 80)

# 다운로드 링크 (Colab용)
if is_colab and df is not None:
    print("\n📥 파일 다운로드:")
    from google.colab import files
    try:
        files.download(Config.OUTPUT_FILENAME)
        print(f"✓ {Config.OUTPUT_FILENAME} 다운로드 시작")
    except:
        print(f"⚠️  수동으로 다운로드하세요: 왼쪽 파일 탭에서 {Config.OUTPUT_FILENAME}")

print("\n✅ 프로그램 실행 완료!")

🚢 트레드링스 선박 스케줄 크롤러
Step 0: 환경 설정 중...

[필수 라이브러리 설치]
📦 selenium 설치 중...
✓ selenium 설치 완료
✓ pandas 이미 설치됨
✓ openpyxl 이미 설치됨
📦 webdriver-manager 설치 중...
✓ webdriver-manager 설치 완료

[Google Colab 환경 감지]
Chrome 및 ChromeDriver 설치 중...
cp: '/usr/lib/chromium-browser/chromedriver' and '/usr/bin/chromedriver' are the same file
✓ Chrome 설치 완료

✓ 환경 설정 완료

✓ 라이브러리 임포트 완료

설정
출발항: Los Angeles
도착항: Busan
기간: 2025-10-20 ~ 2025-11-10
출력 파일: LA_Busan_Schedule_2025-10-20_to_2025-11-10.xlsx

크롤링 시작

웹드라이버 초기화
✓ Chrome 웹드라이버 초기화 완료 (Colab)

트레드링스 접속
URL: https://www.tradlinx.com/ko/schedule
✓ 페이지 로드 완료

검색 조건 설정
✓ 출발지 설정 완료 (셀렉터: input[placeholder*='출발'])
✓ 도착지 설정 완료 (셀렉터: input[placeholder*='도착'])
⚠️  검색 버튼을 찾을 수 없습니다.

스케줄 데이터 추출
⚠️  스케줄 데이터를 찾을 수 없습니다.
웹사이트 구조가 변경되었거나 로그인이 필요할 수 있습니다.


⚠️  실제 웹사이트 크롤링 대신 샘플 데이터를 생성합니다.
(실제 운영 시에는 웹사이트 구조에 맞게 CSS 셀렉터를 수정해야 합니다)

✓ 브라우저 종료

데이터 정렬 및 저장
✓ 16개 스케줄 정렬 완료

데이터 미리보기:
          선사                   선박명    항차          출발항    도착항     LA항 출항일     부산항 도착일 소요기간  

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✓ LA_Busan_Schedule_2025-10-20_to_2025-11-10.xlsx 다운로드 시작

✅ 프로그램 실행 완료!
